# Credit Risk Scoring

<p align="left">
  <img src="https://botsify.com/blog/wp-content/uploads/2024/09/credit-score-1024x636.jpg" width="600">
</p>


## Scenario:  
Imagine we work in a bank and when banks receives application for a loan we need to make sure that in case of approving the loan, client will be able to pay out the debt.  
We want to minimize the risks before providing with the loan and check the probability of the default.  
  
Machine Learning can help us to calculate the risks. To build such model we need only the data about previously approved loans with clients' information and whether the debt has been fully paid out or client failed to pay money back.

## Dataset and Packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

In [3]:
df = pd.read_csv("CreditScoring.csv")
df.columns = df.columns.str.lower()
df.head()

,status,seniority,home,time,age,marital,records,job,expenses,income,assets,debt,amount,price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,1,0,1,36,26,1,1,1,46,107,0,0,310,910


We see that the DataFrame contains the following columns:

- **status** — whether the client managed to repay the loan (1) or not (2);

- **seniority** — length of employment in years;

- **home** — type of home ownership: rent (1), ownership (2), and others;

- **time** — planned loan duration (in months);

- **age** — client’s age;

- **marital_status** — single (1), married (2), etc.;

- **records** — whether the client has any previous records: no (1), yes (2).  
  (The dataset description does not clearly specify what kind of records these are.  
  Given the nature of the project, it is reasonable to assume they refer to records in the bank’s database.)

- **job** — type of employment: full-time (1), part-time (2), and others;

- **expenses** — client’s monthly expenses;

- **income** — client’s monthly income;

- **assets** — total value of the client’s assets;

- **debt** — total amount of outstanding debt;

- **amount** — requested loan amount;

- **price** — price of the item the client wants to purchase.
